In [30]:
# Import Libraries and dependencies
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import yfinance as yf
from finta import TA
import hvplot.pandas
import holoviews as hv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

In [40]:
#Get Nasdaq 100(^NDX) for 1d from yahoo finance for MAX period.
nasdaq100_df = yf.download(tickers='^NDX', period='1y', interval='1h')
display(nasdaq100_df)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2021-01-15 09:30:00-05:00,12895.806641,12938.523438,12767.100586,12799.516602,12799.516602,0
2021-01-15 10:30:00-05:00,12802.329102,12865.330078,12759.160156,12865.271484,12865.271484,95482586
2021-01-15 11:30:00-05:00,12864.475586,12879.758789,12835.425781,12840.352539,12840.352539,61847610
2021-01-15 12:30:00-05:00,12839.710938,12891.900391,12836.959961,12878.650391,12878.650391,48912180
2021-01-15 13:30:00-05:00,12878.438477,12889.433594,12848.815430,12853.837891,12853.837891,56086545
...,...,...,...,...,...,...
2022-01-14 11:30:00-05:00,15482.029297,15577.135742,15471.034180,15524.724609,15524.724609,78214577
2022-01-14 12:30:00-05:00,15522.715820,15539.063477,15416.899414,15433.841797,15433.841797,64339654
2022-01-14 13:30:00-05:00,15432.934570,15541.977539,15419.278320,15491.667969,15491.667969,69756668
2022-01-14 14:30:00-05:00,15489.943359,15593.200195,15489.448242,15573.949219,15573.949219,79557182


In [41]:
# Visualize Close data for nasdaq100_df
close_plt = nasdaq100_df['Close'].hvplot(kind='line', title='Nasdaq 100 Close Prices', xlabel='Date', ylabel='Price ($)', line_color='grey')
close_plt

:Curve   [index]   (Close)

In [42]:
# Set period to be used for trading indicators
period = 20

In [43]:
# Use BBANDS and MFI from TA to get trading indicators for nasdaq100_df
bbands_df = TA.BBANDS(nasdaq100_df, period=period)
mfi_df = pd.DataFrame({'MFI':TA.MFI(nasdaq100_df, period=period)})
display(bbands_df)
display(mfi_df)

,BB_UPPER,BB_MIDDLE,BB_LOWER
2021-01-15 09:30:00-05:00,NaN,NaN,NaN
2021-01-15 10:30:00-05:00,NaN,NaN,NaN
2021-01-15 11:30:00-05:00,NaN,NaN,NaN
2021-01-15 12:30:00-05:00,NaN,NaN,NaN
2021-01-15 13:30:00-05:00,NaN,NaN,NaN
...,...,...,...
2022-01-14 11:30:00-05:00,16090.524509,15758.286914,15426.049320
2022-01-14 12:30:00-05:00,16101.651144,15740.012891,15378.374637
2022-01-14 13:30:00-05:00,16098.375423,15723.082764,15347.790104
2022-01-14 14:30:00-05:00,16086.192957,15709.663330,15333.133703


,MFI
2021-01-15 09:30:00-05:00,NaN
2021-01-15 10:30:00-05:00,NaN
2021-01-15 11:30:00-05:00,NaN
2021-01-15 12:30:00-05:00,NaN
2021-01-15 13:30:00-05:00,NaN
...,...
2022-01-14 11:30:00-05:00,41.351902
2022-01-14 12:30:00-05:00,37.022599
2022-01-14 13:30:00-05:00,36.467684
2022-01-14 14:30:00-05:00,35.636747


In [44]:
#Visualize data for bbands_df and mfi_df
bbands_plt = bbands_df.hvplot(kind="line", xlabel='Datetime', ylabel='Price ($)', title='Bolinger Bands for NASDAQ 100')
mfi_plt = mfi_df.hvplot(kind='line', xlabel='Datetime', ylabel='MFI Values', title='Money Flow Index for NASDAQ 100')
mfi_sell_line = hv.HLine(80).opts(color='red', line_dash='dashed', line_width=2.0)
mfi_buy_line = hv.HLine(20).opts(color='green', line_dash='dashed', line_width=2.0)
display(bbands_plt*close_plt)
display(mfi_plt*mfi_buy_line*mfi_sell_line)

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [index]   (value)
   .Curve.Close :Curve   [index]   (Close)

:Overlay
   .Curve.I  :Curve   [index]   (MFI)
   .HLine.I  :HLine   [x,y]
   .HLine.II :HLine   [x,y]

In [45]:
# Create a copy of nasdaq100_df and concatenate with bbands_df and mfi_df
trading_signals_df = nasdaq100_df.copy()
trading_signals_df = pd.concat([trading_signals_df, bbands_df, mfi_df], axis=1)
display(trading_signals_df)

,Open,High,Low,Close,Adj Close,Volume,BB_UPPER,BB_MIDDLE,BB_LOWER,MFI
2021-01-15 09:30:00-05:00,12895.806641,12938.523438,12767.100586,12799.516602,12799.516602,0,NaN,NaN,NaN,NaN
2021-01-15 10:30:00-05:00,12802.329102,12865.330078,12759.160156,12865.271484,12865.271484,95482586,NaN,NaN,NaN,NaN
2021-01-15 11:30:00-05:00,12864.475586,12879.758789,12835.425781,12840.352539,12840.352539,61847610,NaN,NaN,NaN,NaN
2021-01-15 12:30:00-05:00,12839.710938,12891.900391,12836.959961,12878.650391,12878.650391,48912180,NaN,NaN,NaN,NaN
2021-01-15 13:30:00-05:00,12878.438477,12889.433594,12848.815430,12853.837891,12853.837891,56086545,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-01-14 11:30:00-05:00,15482.029297,15577.135742,15471.034180,15524.724609,15524.724609,78214577,16090.524509,15758.286914,15426.049320,41.351902
2022-01-14 12:30:00-05:00,15522.715820,15539.063477,15416.899414,15433.841797,15433.841797,64339654,16101.651144,15740.012891,15378.374637,37.022599
2022-01-14 13:30:00-05:00,15432.934570,15541.977539,15419.278320,15491.667969,15491.667969,69756668,16098.375423,15723.082764,15347.790104,36.467684
2022-01-14 14:30:00-05:00,15489.943359,15593.200195,15489.448242,15573.949219,15573.949219,79557182,16086.192957,15709.663330,15333.133703,35.636747


In [46]:
# Create a trading algorithm using Bollinger Bands
# Set the Signal column
trading_signals_df["Signal_BB"] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
# trading signal adds one for each buy and doesnt buy more until a sell which resets the trade signal back to 0 and vice verca
for index, row in trading_signals_df.iterrows():
    if (row["Close"] < row["BB_LOWER"]) and (trade_signal < 1):
        trading_signals_df.loc[index, "Signal_BB"] = 1.0
        trade_signal += 1
    elif (row["Close"] > row["BB_UPPER"]) and (trade_signal > 0):
        trading_signals_df.loc[index,"Signal_BB"] = -1.0
        trade_signal = 0

trading_signals_df['Signal_BB'].value_counts()

 0.0    1733
 1.0      18
-1.0      17
Name: Signal_BB, dtype: int64

In [47]:
# Create a trading algorithm using Money Flow Index
# Set Signal column
trading_signals_df['Signal_MFI'] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the MFI is less than the 20 (Oversold)
# where -1 is when the MFI is greater than 80 (Overbought)
# trading signal adds one for each buy and doesnt buy more until a sell which resets the trade signal back to 0 and vice versa
for index, row in trading_signals_df.iterrows():
    if (row['MFI'] > 80) and (trade_signal > 0):
        trading_signals_df.loc[index, 'Signal_MFI'] = -1
        trade_signal = 0
    elif (row['MFI'] < 20) and (trade_signal < 1):
        trading_signals_df.loc[index, 'Signal_MFI'] = 1
        trade_signal += 1
        
trading_signals_df['Signal_MFI'].value_counts()

 0.0    1760
-1.0       4
 1.0       4
Name: Signal_MFI, dtype: int64

In [48]:
def signal_plot(df, signal):
    # Visualize entry position relative to close price
    entry = df[df[signal] == 1.0]["Close"].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price ($)',
        width=1000,
        height=400
    )

    # Visualize exit position relative to close price
    exit = df[df[signal] == -1.0]["Close"].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price ($)',
        width=1000,
        height=400
    )
    
    return entry*exit

display(signal_plot(trading_signals_df, 'Signal_BB')*close_plt.opts(title='Bolinger Bands Trading Strategy')*bbands_plt)

display(signal_plot(trading_signals_df, 'Signal_MFI')*close_plt.opts(title='Money Flow Index Trading Strategy'))

:Overlay
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)
   .Curve.Close      :Curve   [index]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [index]   (value)

:Overlay
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)
   .Curve.Close      :Curve   [index]   (Close)

In [49]:
# Set trading signals by using daily returns
trading_signals_df['Actual_Returns'] = trading_signals_df['Adj Close'].pct_change()
trading_signals_df = trading_signals_df.dropna()
# Initialize the new Signal column
trading_signals_df['Signal_RTN'] = 0.0

# Generate Signal to buy stock long
trading_signals_df.loc[(trading_signals_df['Actual_Returns'] >= 0), 'Signal_RTN'] = 1

# Generate Signal to sell stock short
trading_signals_df.loc[(trading_signals_df['Actual_Returns'] < 0), 'Signal_RTN'] = -1

trading_signals_df['Signal_RTN'].value_counts()

 1.0    941
-1.0    808
Name: Signal_RTN, dtype: int64

In [52]:
# Find the Strategy Returns for the trading strategy. and visualize comparision of actual vs strategy
def compare_returns(df, signal):
    df['Strategy_Returns'] = df['Actual_Returns'] * df[signal].shift()
    return (1 + df[['Actual_Returns','Strategy_Returns']]).cumprod().hvplot(title=f'{signal} Strategy Returns Vs Actual Returns')

compare_returns(trading_signals_df, 'Signal_RTN')

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [26]:
# Set X and y input for NN
X = trading_signals_df[['BB_UPPER', 'BB_MIDDLE', 'BB_LOWER', 'MFI']].shift().dropna().copy()
y = trading_signals_df['Signal_RTN']

display(X.head())
display(y[:5])

,BB_UPPER,BB_MIDDLE,BB_LOWER,MFI
Date,,,,
1985-10-30,117.636228,112.39925,107.162272,57.952953
1985-10-31,118.035265,112.64650,107.257735,63.490767
1985-11-01,118.339686,112.87700,107.414315,62.915519
1985-11-04,118.683748,113.18700,107.690253,67.758641
1985-11-05,118.682431,113.57900,108.475570,71.788094


Date
1985-10-29    1.0
1985-10-30    1.0
1985-10-31   -1.0
1985-11-01    1.0
1985-11-04   -1.0
Name: Signal_RTN, dtype: float64

In [61]:
# Set training start and end dates using DateOffset
# Set offset in years to offset data
offset = 6 
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(years= offset)
print(f'Training Start: {training_begin}, Training End: {training_end}')

Training Start: 1985-10-30 00:00:00, Training End: 1991-10-30 00:00:00


In [62]:
# Set X_train, y_train, X_test, y_test
x_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
x_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [63]:
# Scale X_training and X_testing sets using StandardScaler()/MinMaxScaler()
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
x_train_scaled

array([[-1.98696025, -1.99118389, -1.94370817,  0.0265237 ],
       [-1.97733716, -1.9849392 , -1.94124346,  0.41680908],
       [-1.96999581, -1.97911757, -1.93720086,  0.37626764],
       ...,
       [ 2.35990831,  2.39491225,  2.36984684, -1.65148106],
       [ 2.35804951,  2.39344737,  2.36884193, -1.35847643],
       [ 2.36245437,  2.39561311,  2.36855393, -1.33319197]])

In [64]:
# Create data for using the last 60 signals worth of trading indicators to predict the signal for the next instance
X_train_ar = []
y_train_ar = []

for i in range(60, len(x_train_scaled)):
    X_train_ar.append(x_train_scaled[i-60:i])
    y_train_ar.append(y_train[i])

In [75]:
# Create data array for testing data
X_test_ar = []
y_test_ar = y_test[60:]

for i in range(60, len(x_test_scaled)):
    X_test_ar.append(x_test_scaled[i-60:i])

In [76]:
# Convert data to numpy arrays and then check shape for LSTM(3 dimensional)
X_train_ar, y_train_ar = np.array(X_train_ar), np.array(y_train_ar)
X_train_ar.shape

(1458, 60, 4)

In [77]:
X_test_ar= np.array(X_test_ar)
X_test_ar.shape

(7551, 60, 4)

In [80]:
# Set up the Nueral Network model
nn = Sequential()
nn.add(LSTM(50, activation="tanh",return_sequences=True, input_shape=(X_train_ar.shape[1], X_train_ar.shape[2])))
nn.add(LSTM(50, activation="tanh", return_sequences=False))
nn.add(Dense(25, activation=None))
nn.add(Dense(1, activation=None))

In [81]:
# Compile the Nueral Network model
nn.compile(optimizer='adam', loss='squared_hinge')

In [82]:
# Fit the model with training data
nn.fit(X_train_ar, y_train_ar, batch_size=1, epochs=1)

1458/1458 [==============================] - 30s 19ms/step - loss: 1.0065


In [83]:
predictions = nn.predict(X_test_ar)
predictions

array([[0.10933544],
       [0.10738653],
       [0.10565941],
       ...,
       [0.04367298],
       [0.04393049],
       [0.04415104]], dtype=float32)

In [91]:
pred_df = pd.DataFrame({'Actual': y_test_ar, 'Predictions':np.reshape(predictions, (len(predictions), 1))})
pred_df

ValueError: Data must be 1-dimensional